# 1 加载数据项

In [284]:
import pandas as pd


train_df = pd.read_table('../../round1_ijcai_18_train_20180301.txt',sep=' ')
test_df = pd.read_table('../../round1_ijcai_18_test_a_20180301.txt',sep=' ')

# 线下线上数据统一进行特征处理
test_df['is_trade'] = -1
total_df = pd.concat([train_df, test_df], axis=0)

# 2 预处理数据

In [287]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数
# date最终不使用，直接用day(第 0 - 7 天)

from sklearn import preprocessing
from datetime import datetime
le = preprocessing.LabelEncoder()

def extract_date(x):
    d = datetime.fromtimestamp(x)
    return d.strftime('%Y-%m-%d')
def extract_weekday(x):
    d = datetime.fromtimestamp(x)
    return d.weekday()
def extract_hour(x):
    d = datetime.fromtimestamp(x)
    return d.hour

total_df['day'] = le.fit_transform(total_df['context_timestamp'].apply(lambda x: extract_date(x)))
total_df['weekday'] = total_df['context_timestamp'].apply(lambda x: extract_weekday(x))
total_df['hour'] = total_df['context_timestamp'].apply(lambda x: extract_hour(x))


# 3 复杂类型特征处理

In [253]:
# train_df['item_category_list_len'] = train_df['item_category_list'].apply(lambda x: len(x.split(';')))
# train_df['item_property_list_len'] = le.fit_transform(\
#             pd.cut(train_df['item_property_list']\
#                    .apply(lambda x: len(x.split(';'))), bins=10, retbins=False\
#                   )\
#                                                      )


In [289]:
#简化list等复杂类型的结构
#item_category_list全展开
#item_property_list全展开取频率>0.05的数据
#predict_category_property会计算跟item prop和cate的余弦相似度
#具体是把两个list的数据拼成cate:-1和cate:prop两种方式拼成一个字符串再跟predict_category_property的数据计算相似度

def get_icl_map(df):
    print("get_icl_map ... ")
    dfX = df.copy()
    dfX = dfX['item_category_list'].str.split(';', expand=True)

#     s = sorted(list(dfX[0].unique()) + list(dfX[1].unique()) + list(dfX[2].unique()))
    m = {}
    for i in dfX[0].unique():
        if i == None:
            continue
        m[i] = "1"
    
    for i in dfX[1].unique():
        if i == None:
            continue
        m[i] = "2"
    
    for i in dfX[2].unique():
        if i == None:
            continue
        m[i] = "3"
    return m

def get_ipl_map(df):
    print("get_ipl_map ... ")
    df1 = df.copy()
    dfX = df1.copy()['item_property_list'].str.split(';')
    dfX = pd.DataFrame(dfX)
    
    m = collections.defaultdict(float)
    idx = 0
    for _, row in dfX.iterrows():
        for i in row[0]:
            m[i] += 1
    
    ll = len(dfX)
    for k,v in m.items():
        m[k] = v / ll
    return m

def process_complex_types(dfX, icl_map, ipl_map):
    def filter_unless_cate(arr):
        ret = []
        for i in arr:
            if i in icl_map:
                ret.append(i)
        if len(ret) == 0:
            return None
        else:
            return ret
    
    def filter_unless_prop(arr):
        ret = []
        for i in arr:
            freq = ipl_map.get(i, 0.)
            if freq > 0.05:
                ret.append(i)
        if len(ret) == 0:
            return None
        else:
            return ret
    
    #{cate}:-1命中则为1分
    #{cate}:{prop}命中则为2分
    #后期优化权重
    def inner_product_recall_items(line):
        line = line.split("|")
        item_category_list = line[0].split(";")
        item_property_list = line[1].split(";")
        
        whole_combines = {}
        for cate in item_category_list:
            tmp = cate+":"+"-1"
            whole_combines[tmp] = 1
            for prop in item_property_list:
                tmp = cate+":"+prop
                whole_combines[tmp] = 2
        
        
                
        predict_category_property = line[2].split(";")
        product = 0.
        item_vec_len = math.sqrt(len(whole_combines))
        user_vec_len = math.sqrt(len(predict_category_property))
        for item in predict_category_property:
            #x1 == 1
            #y1 == 1
            #x1*y1 == 1
            #x2 == 0
            #y2 == 1
            #x2*y2 == 0
            #所以product由x决定 += 1/0
            product += whole_combines.get(item, 0)
        
        return product/(item_vec_len*user_vec_len)
            
        
            
        
        
        
    print("processing predict_category_property ...")
#     dfX['predict_category_property'] = dfX['predict_category_property'].str.split(';').map(lambda x: [i.split(":")[0] for i in x]).map(filter_unless_cate)
    
    dfX['predict_richness'] =  dfX['predict_category_property'].map(lambda x: 0 if len(x.strip()) == 0 else len(x.split(";")))
    dfX['predict_category_property'] = dfX['item_category_list']+"|"+dfX['item_property_list']+"|"+dfX['predict_category_property']
    dfX['predict_category_property'] = dfX['predict_category_property'].map(inner_product_recall_items)
    
    print("processing item_property_list ...")
    dfX['item_property_richness'] = dfX['item_property_list'].map(lambda x: 0 if len(x.strip()) == 0 else len(x.split(";")))
    dfX['item_property_list'] = dfX['item_property_list'].str.split(';').map(filter_unless_prop)
    
    print("processing item_category_list ...")
    dfX['item_category_list'] = dfX['item_category_list'].str.split(';')
    
#     print "generating item_category_list01 - item_category_list03 ..."
#     dfX['item_category_list01'] = dfX['item_category_list'].map(lambda x:x[0] if x != None and len(x) > 0 else None)
#     dfX['item_category_list02'] = dfX['item_category_list'].map(lambda x:x[1] if x != None and len(x) > 1 else None)
#     dfX['item_category_list03'] = dfX['item_category_list'].map(lambda x:x[2] if x != None and len(x) > 2 else None)
    
    return dfX



# aaa = process_complex_types(train_df.copy(), get_icl_map(train_df), get_ipl_map(train_df))

total_df = process_complex_types(total_df, get_icl_map(total_df), get_ipl_map(total_df))

get_icl_map ... 


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x10c5f6a58>>
Traceback (most recent call last):
  File "/Users/prehawkmac/anaconda3/lib/python3.6/site-packages/xgboost/core.py", line 368, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


get_ipl_map ... 
processing predict_category_property ...
processing item_property_list ...
processing item_category_list ...


# 转化属性增加trade_rate + trade_pv（按前一天）

In [290]:
# 统计信息附加

def calcTVTransform(df, key, key_y, filter_src, filter_dst, smoothing = 10, mean0=None):
    if mean0 is None:
        #计算目标的平均值做平缓用
        mean0 = df.ix[filter_src, key_y].mean()
        print("mean0:", mean0)
    
    #取出key的所有值
    df['_key1'] = df[key].astype('category').values.codes
    
    
    #取出用于计算的源（后面聚合掉就没有顺序可言了）
    df_key1_y = df.ix[filter_src, ['_key1', key_y]]
    
    #根据key的取值去聚合key_y的总数和总和，用户计算rate和count
    grp1 = df_key1_y.groupby(['_key1'])
    sum1 = grp1[key_y].aggregate(np.sum)
    cnt1 = grp1[key_y].aggregate(np.size)
    
    vn_sum = 'sum_' + key
    vn_cnt = 'cnt_' + key
    
    #取出dst（带序列）的所有key
    v_codes = df.ix[filter_dst, '_key1']
    
    #得到_sum,_cnt，按dst的序列
    _sum = sum1[v_codes].values
    _cnt = cnt1[v_codes].values
    _cnt[np.isnan(_sum)] = 0    
    _sum[np.isnan(_sum)] = 0
    
    r = {}
    r['exp'] = (_sum + smoothing * mean0)/(_cnt + smoothing)
    r['cnt'] = _cnt
    return r


In [291]:
#计算前一天的交易率set到下一天，第0天用回自己
# tmp = train_df.copy()
tmp = total_df

add_count = False
# window = 2

exp = "exp_d_"
cnt = "cnt_d_"


exp_numerical = {}
cnt_numerical = {}

#此处应该处理按天为梯度的数据
for k in ['item_price_level',
'item_sales_level',
'item_collected_level',
'item_pv_level',
'user_gender_id',
'user_age_level',
'user_occupation_id',
'user_star_level',
'context_page_id',
'shop_review_num_level',
'shop_star_level']:
    exp_k = exp+k
    cnt_k = cnt+k
    for day in range(0,8):
#         start_d = max(day - window, 0)
#         end_d = max(day - 1,0)
        cal_day = max(day - 1, 0)
        set_day = day

        # print("cal %s trade_rate cnt %s set to %s" % (k,cal_day, set_day))
        
        #start_d - day(不含day)用于计算，结果赋值到day上
        days1 = (tmp.day.values == cal_day)
        days2 = (tmp.day.values == set_day)
        ret = calcTVTransform(tmp, k, 'is_trade', days1, days2)
            
        tmp.loc[tmp.day.values == day, exp_k] = ret["exp"]
        
        exp_numerical[exp_k]=1
        if add_count:
            cnt_numerical[cnt_k]=1
            tmp.loc[tmp.day.values == day, cnt_k] = ret["cnt"]


/Users/prehawkmac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


mean0: 0.020020953646445547
mean0: 0.020020953646445547
mean0: 0.019681098532376536
mean0: 0.01936040475528975
mean0: 0.019312068989733002
mean0: 0.018867648350361546
mean0: 0.017276071305058635
mean0: 0.016910189651869526
mean0: 0.020020953646445547
mean0: 0.020020953646445547
mean0: 0.019681098532376536
mean0: 0.01936040475528975
mean0: 0.019312068989733002
mean0: 0.018867648350361546
mean0: 0.017276071305058635
mean0: 0.016910189651869526
mean0: 0.020020953646445547
mean0: 0.020020953646445547
mean0: 0.019681098532376536
mean0: 0.01936040475528975
mean0: 0.019312068989733002
mean0: 0.018867648350361546
mean0: 0.017276071305058635
mean0: 0.016910189651869526
mean0: 0.020020953646445547
mean0: 0.020020953646445547
mean0: 0.019681098532376536
mean0: 0.01936040475528975
mean0: 0.019312068989733002
mean0: 0.018867648350361546
mean0: 0.017276071305058635
mean0: 0.016910189651869526
mean0: 0.020020953646445547
mean0: 0.020020953646445547
mean0: 0.019681098532376536
mean0: 0.019360404755289

In [292]:
exp_numerical = list(exp_numerical.keys())
cnt_numerical = list(cnt_numerical.keys())

print(exp_numerical)
print(cnt_numerical)

['exp_d_item_price_level', 'exp_d_item_sales_level', 'exp_d_item_collected_level', 'exp_d_item_pv_level', 'exp_d_user_gender_id', 'exp_d_user_age_level', 'exp_d_user_occupation_id', 'exp_d_user_star_level', 'exp_d_context_page_id', 'exp_d_shop_review_num_level', 'exp_d_shop_star_level']
[]


# 标记训练用的列

In [293]:
#标记训练数据

target="is_trade"
                          
                          
                        
categorical=[  
#                 'shop_id',
#                 'item_brand_id',
#                 'item_city_id',
                'item_price_level',
                'item_sales_level',
                'item_collected_level',
                'item_pv_level',
                'user_gender_id',
                'user_age_level',
                'user_occupation_id',
                'user_star_level',
                'context_page_id',
                'shop_review_num_level',
                'shop_star_level',
                "weekday",
                "hour"]

numerical=[     'shop_review_positive_rate',
                'shop_score_service',
                'shop_score_delivery',
                'shop_score_description',
                'predict_category_property',
                'predict_richness',
                'item_property_richness'

          ]#+exp_numerical
listype = [
    'item_property_list', 
    'item_category_list'
]

class filter_on_cols:
    def __init__(self, target, categorical, numerical, listype):
        self.target = target
        self.categorical = categorical
        self.numerical = numerical
        self.listype = listype
        

    
#     def get_raw_simple_cols(self):
#         return [self.target]+self.categorical+self.numerical
    def get_raw_target_col(self):
        return self.target
    def get_raw_categorical_cols(self):
        return self.categorical
    
    def get_raw_numerical_cols(self):
        return self.numerical
    def get_raw_listype_cols(self):
        return self.listype
    
    def get_onehoted_cols(self, t, df):
        if t == "listype":
            org_cols = self.listype
        
        if t == "categorical":
            org_cols = self.categorical
            
        ret = []
        for org_col in org_cols:
            for cur_col in list(df.columns):
                if cur_col.find("*ONEHOT*_") != -1 and cur_col.find(org_col) != -1:
                    ret.append(cur_col)
        
        return ret
                    
                    
            
            

filter_on_cols = filter_on_cols(target, categorical, numerical, listype)

# 平滑处理连续型数据最后对复杂类型做onehot编码

In [294]:
#数据的预处理
#double系列(例如好评率)的数据存在一个问题，没有考虑到评论量，所以乘了一个评论level，类似 好评率*评论数(分桶了) = 好评量(分桶了)
#fillempty把-1设置成-0.01，配合mov2pos使用
#mov2pos把有<0的数据都集体-min，如果一个数据是0-1，有-1出现的时候，上面设置成了-0.01，所以这列数据真实是-0.01 - 1,然后我会集体-(-0.01)
#norm用了最大值最小值norm
%run FeatureProcess.py

total_df = total_df.copy()
featProc = FeatureProcess(target=target, categorical=categorical, numerical=numerical, listype = listype)

total_df["shop_review_num_level"] = total_df["shop_review_num_level"]
total_df["shop_review_positive_rate"] = total_df["shop_review_positive_rate"]*total_df["shop_review_num_level"]
total_df["shop_score_service"] = total_df["shop_score_service"]*total_df["shop_review_num_level"]
total_df["shop_score_delivery"] = total_df["shop_score_delivery"]*total_df["shop_review_num_level"]
total_df["shop_score_description"] = total_df["shop_score_description"]*total_df["shop_review_num_level"]

total_df = featProc.fillempty(total_df, -0.01)
total_df = featProc.mov2pos(total_df)
total_df = featProc.norm(total_df)
# train_df[filter_on_cols.get_raw_numerical_cols()].describe()


# train_df[filter_on_cols.get_raw_numerical_cols()].describe()

# 添加id分布穿越特征

In [295]:
# 针对有意义的id进行分桶
# item_city_id, item_brand_id, shop_id, item_id
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

def bucker(se, bins, col):
    """获得某一列的分桶标识"""
    df = pd.DataFrame(pd.cut(se, bins=bins, retbins=False))
    df.columns = [col]
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    return df

# 先构造统计表格
tr_city_counts = bucker(train_df['item_city_id'].value_counts(), 10, 'item_city_id_buck')
te_city_counts = bucker(test_df['item_city_id'].value_counts(), 10, 'item_city_id_buck')

tr_brand_counts = bucker(train_df['item_brand_id'].value_counts(), 10, 'item_brand_id_buck')
te_brand_counts = bucker(test_df['item_brand_id'].value_counts(), 10, 'item_brand_id_buck')

tr_shop_counts = bucker(train_df['shop_id'].value_counts(), 10, 'shop_id_buck')
te_shop_counts = bucker(test_df['shop_id'].value_counts(), 10, 'shop_id_buck')

tr_item_counts = bucker(train_df['item_id'].value_counts(), 10, 'item_id_buck')
te_item_counts = bucker(test_df['item_id'].value_counts(), 10, 'item_id_buck')

In [301]:
# 应用到原始df
try:
    del total_df['item_city_id_buck'], total_df['item_brand_id_buck'], total_df['shop_id_buck'], total_df['item_id_buck']
except:
    pass

total_df = total_df.join(tr_city_counts, on='item_city_id')
total_df = total_df.join(tr_brand_counts, on='item_brand_id')
total_df = total_df.join(tr_shop_counts, on='shop_id')
total_df = total_df.join(tr_item_counts, on='item_id')


total_df = total_df.join(te_city_counts, on='item_city_id', rsuffix='_drop')
total_df = total_df.join(te_brand_counts, on='item_brand_id', rsuffix='_drop')
total_df = total_df.join(te_shop_counts, on='shop_id', rsuffix='_drop')
total_df = total_df.join(te_item_counts, on='item_id', rsuffix='_drop')
total_df = total_df.drop(['item_city_id_drop', 'item_brand_id_drop', 'shop_id_drop', 'item_id_drop'], axis=1)


In [302]:
#onehot没啥好说的，就是onehot，toOneHotList实现有点不同分开了而已
total_df = featProc.toOneHot(total_df)


# train_df = featProc.toOneHotList(train_df)

# train_df = featProc.cacheRun(featProc.toOneHot, train_df)
# for c in train_df.columns:
#     print(c)



toOneHot ...


# 指定需要用到的列

In [335]:
train_df = total_df[total_df['is_trade']>=0]
test_df = total_df[total_df['is_trade']<0]

non_feat_columns = ['context_timestamp', 'instance_id', 'item_id', 'user_id', 'shop_id', 'item_brand_id', 'context_id',
                   'item_city_id_buck', 'item_brand_id_buck', 'shop_id_buck', 'item_id_buck', 
                   'item_property_list', 'item_category_list', 
                   'is_trade']

D = train_df['context_timestamp']
X = train_df.drop(non_feat_columns, axis=1).values
y = train_df[['is_trade']].values.ravel()

X_online = test_df.drop(non_feat_columns, axis=1).values
print(D.shape, X.shape, y.shape, X_online.shape)

(478138,) (478138, 242) (478138,) (18371, 242)


In [337]:
# 训练模型
# from sklearn.model_selection import GridSearchCV as GSCV   #Perforing grid search
# from sklearn.metrics import make_scorer
from sklearn.metrics import log_loss
import xgboost

%run ../util/time_series_split.py

xgb = xgboost.XGBClassifier(n_jobs=5)

m = None

# 出离线结果, 留空最后一天数据
# dtsv = DateTimeSplit(dateSeries=D, fmt="%Y-%m-%d", n=1)
# for train_i, test_i in dtsv.split(X, y):
#     X_train, y_train = X[train_i], y[train_i]
#     X_test, y_test = X[test_i], y[test_i]
#     m = xgb.fit(X_train, y_train)
#     print(log_loss(y_test, m.predict_proba(X_test)))

# 不出离线结果, 全量数据
m = xgb.fit(X, y)
    

In [338]:
# 输出比赛格式
result = pd.DataFrame()
result['instance_id'] = test_df['instance_id']
result['predicted_score'] = pd.DataFrame(m.predict_proba(X_online))[1].values
result.to_csv('20180321_xgb.csv', sep = ' ', header=True, index = False)


In [ ]:
#这块一般不怎么需要改了，拿最后一天做验证
#注意下面验证版和测试版，测试版是我们自己玩的，验证版是扔线上的，通过注释不同的跑就行了


def ffea(on_cols):
    ret = {}
    for v in on_cols:
        if v == "user_occupation_id":
            continue
        
        ret[v] = 1
    return ret.keys()

# %run BaseFrame.py
% run ../util/time_series_split.py

tmp_train_df = train_df.copy().fillna(0)


categorical_one_hot_cols = filter_on_cols.get_onehoted_cols("categorical", tmp_train_df)
listype_one_hot_cols = filter_on_cols.get_onehoted_cols("listype", tmp_train_df)

categorical_cols = filter_on_cols.get_raw_categorical_cols()
listype_cols = filter_on_cols.get_raw_listype_cols()
numerical_cols = filter_on_cols.get_raw_numerical_cols()
target_col = filter_on_cols.get_raw_target_col()

print("org dataset = %s" % len(tmp_train_df))
print(tmp_train_df["day"].value_counts())

d_train_indices = None

class DateTimeSplit2:

    def __init__(self, dataSeries, gap):
        self.D = dataSeries 
        self.gap = gap

    def get_n_splits(self, X, y, groups=None):
        return 1

    def split(self, X, y, groups=None):
        # X, y = indexable(X, y)
        train_indices = self.D[self.D['day']<self.gap].reset_index().index.values
        test_indices  = self.D[self.D['day']==self.gap].reset_index().index.values
        #print(train_indices)
        #print(X[np.array(train_indices)].shape, X[np.array(test_indices)].shape)
        yield np.array(train_indices), np.array(test_indices)


from sklearn.model_selection import GridSearchCV as GSCV   #Perforing grid search
from sklearn.metrics import make_scorer
from sklearn.metrics import log_loss
import xgboost


X_train = tmp_train_df[on_cols].copy()
del X_train['is_trade']
y_train = tmp_train_df['is_trade'].copy()
assert(X_train.shape[0] == y_train.shape[0])
assert(X_train.shape[0] == tmp_train_df.shape[0])

dtsv = DateTimeSplit2(tmp_train_df, 6)
score_func = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

parameters = {
  'nthread':[7], #when use hyperthread, xgboost may become slower
  'learning_rate': [0.1], #so called `eta` value
  'max_depth': [5,6,7,8,9,10],
  'n_estimators': [32,64,100],
  'reg_lambda': [0]
}

xgb = xgboost.XGBClassifier()
gs = GSCV(xgb, parameters, scoring=score_func, cv=dtsv)

on_cols = [target_col,"day"]+categorical_cols+numerical_cols


gs.fit(X_train.values, y_train.values)